In [1]:
import cv2

In [2]:
def preprocess_frame(frame, blur_kernel_size=(5, 5)):
    """Preprocess the frame by converting to grayscale and applying Gaussian blur."""
    try:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    except cv2.error as e:
        print(f"Error converting frame to grayscale: {e}")
        print(f"Frame shape: {frame.shape}, Frame type: {type(frame)}")
        return None
    
    frame = cv2.GaussianBlur(frame, blur_kernel_size, 0)

    return frame

In [3]:
from PIL import Image
import imagehash

def compute_frame_phash(gray_frame):
    # Convert to PIL Image
    pil_image = Image.fromarray(gray_frame)
    
    # Compute perceptual hash
    phash = imagehash.phash(pil_image, hash_size=16, highfreq_factor=16)
    
    return phash

In [4]:
def get_frame_hashes(video_path):
    cap = cv2.VideoCapture(video_path)
    phash=[]

    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f'FPS : {fps:0.2f}')

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = preprocess_frame(frame)
        # frames.append(frame)

        phash.append(compute_frame_phash(frame))

    cap.release()
    return phash

In [5]:
VIDEO_ID="video1"

In [31]:
# phash1 = get_frame_hashes('outputMerged.mp4') # previousVideo
# phash1= get_frame_hashes('added_15_frames.mp4')
phash1= get_frame_hashes('main_edited_copy_2.mp4')
# Will be stored in db or text file.

# TODO
# phash1 = get_frame_hashes('outputMerged.mp4')
# Here, we will fetch the previous frame hashes txt file. instead of processing the whole video.
# and then store it in the phash1 array

FPS : 30.00


In [24]:
# Will be done while initial uploading.

with open(f'{VIDEO_ID}/previous_hashes.txt', 'w') as file:
    for item in phash1:
        file.write(str(item) + "\n")
        
    #TODO
    # Should also upload to the server?

In [35]:
# Will be done while initial uploading.
fps=30
previous_length_of_streams=[]
i=len(phash1)

while i>=0 and i//(fps*10)!=0:
    previous_length_of_streams.append(fps*10)
    i-=(fps*10)

if i%(fps*10)!=0:
    previous_length_of_streams.append(i%(fps*10))

print(previous_length_of_streams)

[300, 300, 85]


In [36]:
with open(f'{VIDEO_ID}/previous_length_of_streams.txt', 'w') as file:
    for item in previous_length_of_streams:
        file.write(str(item) + "\n")

In [37]:
with open(f'{VIDEO_ID}/previous_length_of_streams.txt', 'r') as file:
    previous_length_of_streams = [int(line.strip())  for line in file]

In [38]:
with open(f'{VIDEO_ID}/previous_hashes.txt', 'r') as file:
    # Read all lines and convert them to integers
    phash1 = [imagehash.hex_to_hash(line.strip())  for line in file]

In [48]:
# EDITED_VIDEO_NAME='deleted_15_frames.mp4'
EDITED_VIDEO_NAME='added_15_frames.mp4'
phash2= get_frame_hashes(EDITED_VIDEO_NAME)
# Second video (edited version)

FPS : 30.00


In [40]:
def is_significantly_different(hash1, hash2, threshold=5):
    return hash1 - hash2 > threshold

In [49]:
# Detect changes
i,j=0,0
deleted=[]
added=[]
changedFrames=[]
while i<len(phash1) and j<len(phash2):
    currphash1=phash1[i]
    currphash2=phash2[j]
    if is_significantly_different(currphash1, currphash2):
        flag=True
        if (len(phash1)!=len(phash2)):
            for k in range(1,60):
                if i + k >= len(phash1):
                    break
                newphash1=phash1[i+k]
                if newphash1==currphash2:
                    if k+1<15: # doubt-----------------------------------------------------------
                        break
                    print(i, i+k, i/30, "-", (i+k)/30)
                    deleted.append((i,i+k-1))
                    i=i+k
                    print(i/30)
                    flag=False
                    break
            
            if flag:
                for k in range(1,60):
                    if j + k >= len(phash2):
                        break
                    newphash2=phash2[j+k]
                    if currphash1==newphash2:
                        if k+1<15: # doubt-----------------------------------------------------------
                            break
                        print(j, j+k, j/30, "-", (j+k)/30)
                        added.append((j,j+k-1))
                        j=j+k
                        print(j/30)
                        flag=False
                        break
                
        if flag:
            changedFrames.append(j)
            
    i+=1
    j+=1
    
print(deleted)
print(added)
print(changedFrames)

165 180 5.5 - 6.0
6.0
[]
[(165, 179)]
[]


In [50]:
changedSegments=set()

In [51]:
new_length_of_streams=previous_length_of_streams[:]

# For deleted Frames
for deletedFrame in deleted:
    start=deletedFrame[0]
    end=deletedFrame[1]

    index=start//(fps*10)
    changedSegments.add(index)
    new_length_of_streams[index]-=end-start+1

print(new_length_of_streams)
print(changedSegments)

[300, 300, 85]
set()


In [52]:
# For added frames
for addedFrame in added:
    start=addedFrame[0]
    end=addedFrame[1]

    index=start//(fps*10)
    changedSegments.add(index)
    print(index, previous_length_of_streams[index])
    new_length_of_streams[index]+=end-start+1

print(new_length_of_streams)
print(changedSegments)

0 300
[315, 300, 85]
{0}


In [18]:
for frame in changedFrames:
    index=frame//300
    changedSegments.add(index)

print(new_length_of_streams)
print(changedSegments)

[300, 300, 70]
set()


In [19]:
import os
import subprocess

# Function to convert frames to time (seconds)
def frames_to_time(frame_count, fps):
    return round(frame_count / fps, 3)  # Round to 3 decimal places for better precision

# Function to generate HLS segment for each frame range
def generate_hls_segment(video_file, start_time, duration, output_dir, segment_num):
    segment_path = os.path.join(output_dir, f"segment_{segment_num}.ts").replace("\\", "/")
    
    cmd = [
        "ffmpeg",
        "-y",
        "-ss", str(start_time),  # Start time
        "-i", video_file,        # Input video file
        "-t", str(duration),     # Duration of the segment
        "-c:v", "libx264",      # Re-encode video with libx264
        "-c:a", "aac",          # Re-encode audio with AAC
        "-strict", "experimental",  # Allow experimental codecs
        "-force_key_frames", "expr:gte(t,n_forced*1)",  # Force keyframes every second
        "-f", "mpegts",         # Set output format to .ts
        segment_path             # Output .ts file
    ]
    
    # Run the command
    try:
        subprocess.run(cmd, check=True)
        return segment_path
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e}")
        return None

# Function to generate a playlist for all segments
def generate_playlist(segment_durations, final_output_m3u8):
    with open(final_output_m3u8, 'w') as outfile:
        outfile.write('#EXTM3U\n')  # HLS playlist header
        
        for idx, duration in enumerate(segment_durations):
            # Add segment info to playlist
            outfile.write(f'#EXTINF:{duration},\n')
            outfile.write(f"segment_{idx}.ts\n")
            outfile.write('#EXT-X-TARGETDURATION:' + str(max(segment_durations)) + '\n')  

# Main function to process the video based on frame ranges
def create_combined_hls_stream(video_file, frame_ranges, fps, output_dir, final_output_m3u8):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    start_frame = 0
    segment_durations = []

    # Loop through each frame range to create segments
    for idx, frame_count in enumerate(frame_ranges):
        start_time = frames_to_time(start_frame, fps)
        duration = frames_to_time(frame_count, fps)
        
        # Generate HLS segment for this frame range
        segment_path = generate_hls_segment(video_file, start_time, duration, output_dir, idx)
        
        if segment_path:
            segment_durations.append(duration)
        
        # Update the start frame for the next segment
        start_frame += frame_count

    # Generate a playlist with exact durations
    generate_playlist(segment_durations, final_output_m3u8)

In [47]:
output_dir=VIDEO_ID
final_output_m3u8 = os.path.join(output_dir, "playlist.m3u8").replace("\\", "/")
print(new_length_of_streams)
create_combined_hls_stream(EDITED_VIDEO_NAME, new_length_of_streams, fps, output_dir, final_output_m3u8)

[285, 300, 85]


ffmpeg version 6.0-6ubuntu1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-4ubuntu3)
  configuration: --prefix=/usr --extra-version=6ubuntu1.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librist --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libtwolame --enable-libvidstab 

In [21]:
import boto3
import os
from dotenv import load_dotenv
load_dotenv()

s3_client = boto3.client(
    's3',
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_ACCESS_KEY_SECRET"),
    region_name="us-east-1"
)

def upload_segment(segment_number):
    print("Upload file",segment_number)
    # S3 client setup with credentials
    
    # Construct the file name based on segment number
    file_name = f"segment_{segment_number}.ts"
    file_path = os.path.join(output_dir, file_name)
    print(file_path)
    # Check if the file exists
    if not os.path.isfile(file_path):
        print(f"File {file_name} does not exist in {output_dir}")
        return
    
    try:
        # Upload to S3
        s3_client.upload_file(file_path, f"videohlsstreams", f"{VIDEO_ID}/{file_name}")
        print(f"Successfully uploaded {file_name} to S3")
    except Exception as e:
        print(f"Failed to upload {file_name}: {e}")

In [22]:
def create_bucket_if_not_exists(bucket_name, region="us-east-1"):
    try:
        # Check if bucket exists
        s3_client.head_bucket(Bucket=bucket_name)
        print(f"Bucket '{bucket_name}' already exists.")
    except s3_client.exceptions.ClientError as e:
        error_code = int(e.response['Error']['Code'])
        if error_code == 404:
            # If bucket does not exist, create it
            print(f"Bucket '{bucket_name}' does not exist. Creating now...")
            if region == "us-east-1":
                s3_client.create_bucket(Bucket=bucket_name)
            else:
                s3_client.create_bucket(
                    Bucket=bucket_name,
                    CreateBucketConfiguration={'LocationConstraint': region}
                )
            print(f"Bucket '{bucket_name}' created successfully.")
        else:
            raise e

# create_bucket_if_not_exists(VIDEO_ID)

for segment in changedSegments:
    upload_segment(segment)

INIT=True
if INIT:
    for i in range(0, len(new_length_of_streams)):
        upload_segment(i)

file_name = "playlist.m3u8"
file_path = os.path.join(output_dir, file_name)
print(file_path)
s3_client.upload_file(file_path, f"videohlsstreams", f"{VIDEO_ID}/{file_name}")
print("HLS file uploaded")

Upload file 0
video1/segment_0.ts
Failed to upload segment_0.ts: Could not connect to the endpoint URL: "https://videohlsstreams.s3.amazonaws.com/video1/segment_0.ts"
Upload file 1
video1/segment_1.ts


KeyboardInterrupt: 

In [88]:

# For merging all the .ts file into mp4 again
# To be done when merging

# Directory containing your .ts files
def create_video_list_file(videoId):
    # Create the file list
    directory = VIDEO_ID
    with open(f'{videoId}/segments.txt', 'w') as f:
        for filename in sorted(os.listdir(directory)):
            if filename.endswith('.ts') and not filename.__contains__("merged"):
                f.write(f"file '{os.path.join(directory, filename)}'\n")
                
def concatenate_ts_segments(video_list_file, videoId):
    """
    Use FFmpeg to concatenate .ts segments into a single .ts file.
    """
    # Create the video list file
    # video_list_file = f'{videoId}.txt'
    # create_video_list_file(videoId)

    if os.path.isfile(f"{videoId}/merged.ts"):
        os.remove(f"{videoId}/merged.ts")
        print(f"Removed existing file: {videoId}/merged.ts")

    # FFmpeg command to concatenate .ts files
    cmd = [
        'ffmpeg', '-f', 'concat', '-safe', '0',
        '-i', video_list_file,  # Input file list
        '-c', 'copy',           # Copy streams without re-encoding
        f'{videoId}/merged.ts'          # Output file
    ]

    # Run the FFmpeg command
    try:
        subprocess.run(cmd, check=True)
        print(f"Concatenation successful! Output saved as {videoId}.ts")
    except subprocess.CalledProcessError as e:
        print(f"Error during concatenation: {e}")

In [90]:
def convert_ts_to_mp4(input_ts_file, output_mp4_file):
    if os.path.isfile(f"{VIDEO_ID}/final.mp4"):
        os.remove(f"{VIDEO_ID}/final.mp4")
        print(f"Removed existing file: {VIDEO_ID}/final.mp4")

    """
    Convert .ts file to .mp4 using FFmpeg.
    """
    cmd = [
        'ffmpeg', '-i', input_ts_file,  # Input .ts file
        '-c', 'copy',                   # Copy video and audio streams without re-encoding
        output_mp4_file                 # Output .mp4 file
    ]

    # Run the FFmpeg command
    try:
        subprocess.run(cmd, check=True)
        print(f"Conversion successful! Output saved as {output_mp4_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error during conversion: {e}")

In [91]:
create_video_list_file(VIDEO_ID)
concatenate_ts_segments(f'{VIDEO_ID}.txt', VIDEO_ID)
convert_ts_to_mp4(f'{VIDEO_ID}/merged.ts', f'{VIDEO_ID}/final.mp4')

Removed existing file: video1/merged.ts


ffmpeg version 6.0-6ubuntu1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-4ubuntu3)
  configuration: --prefix=/usr --extra-version=6ubuntu1.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librist --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libtwolame --enable-libvidstab 

Concatenation successful! Output saved as video1.ts
Removed existing file: video1/final.mp4


ffmpeg version 6.0-6ubuntu1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-4ubuntu3)
  configuration: --prefix=/usr --extra-version=6ubuntu1.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librist --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libtwolame --enable-libvidstab 

Conversion successful! Output saved as video1/final.mp4


frame=  670 fps=0.0 q=-1.0 Lsize=    4895kB time=00:00:22.39 bitrate=1790.9kbits/s speed= 131x    
video:4681kB audio:195kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.393303%
